# Authorship verification for education

## Classifying inconsistent writing style using known corpus

#### Patrick Steeves

 <br><br>

# 1) Data

Subset of Reuteurs RCV1 corpus (famous dataset of Reuteurs articles used a lot for NLP research)

In [1]:
import os
import pandas as pd
import re

In [2]:
path = "C:\\Users\\patri\\Documents\\GW\\2018Spring\\Hackathon"
os.chdir(path)

authors = os.listdir('.\\C50train')   # Data was separated into folders by author name

articles = pd.DataFrame({'author':[],'text':[]})

for author in authors:
    titles1 = os.listdir('.\\C50train\\'+author)
    for title in titles1:
        with open('.\\C50train\\'+author+'\\'+title) as f:
            text = re.sub(r'http\S+','',f.read()).lower()
            if len(text) > 1500:
                articles = articles.append({'author':author,'text':text}, ignore_index=True)

    titles2 = os.listdir('.\\C50test\\'+author)
    for title in titles2:
        with open('.\\C50test\\'+author+'\\'+title) as f:
            text = re.sub(r'http\S+','',f.read()).lower()
            if len(text) > 1500:
                articles = articles.append({'author':author,'text':text}, ignore_index=True)

In [3]:
articles.to_csv(path+'\\articles.csv', index_label = 'ID')   # Save concatenated file locally

<br><br>

# 2) Utilizing Google Cloud

I loaded my dataset into the cloud using the command line

<img src = "load_data.PNG">

The data is now stored on the Google Cloud Platform. We can leave it there and grab it whenever using the code below at a later time if needed

In [ ]:
# Pull data back from the cloud
from google.cloud import bigquery
auth_file = "C:\\Users\\patri\\Documents\\GW\\2018Spring\\Hackathon\\GCP API\\author-identification-dd17869ab683.json"
client = bigquery.Client.from_service_account_json(auth_file)

pull_job = client.query("""
    SELECT * 
    FROM corpus.articles
    """)

articles = pull_job.result().to_dataframe()

In [4]:
# Combine all articles by author into one big writing sample
articles_aggr = articles.groupby(['author'])['text'].apply(lambda x: ''.join(x)).reset_index()

In [5]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import ngrams
from nltk.corpus import stopwords
import string
from collections import Counter

w_tokenizer = RegexpTokenizer('\w+')

Extract meaningful features to feed into model

In [6]:
def extract_features(df):
    # Tokenize text into sentences and words
    df['sentences'] = df.text.apply(lambda x: sent_tokenize(x))
    df['words'] = df.text.apply(lambda x: w_tokenizer.tokenize(x))
   
    # Compute average word count per sentence, frequency of punctuation, average word length, ratio of unique words 
    df['sent_wc'] = df.sentences.apply(lambda x: sum([len(w_tokenizer.tokenize(sentence)) for sentence in x]) / len(x))
    df['sent_punc'] = df.sentences.apply(lambda x: len([c for c in ''.join(x) if c in set(string.punctuation)]) / len(''.join(x)))
    df['avg_wlength'] = df.words.apply(lambda x: sum([len(w) for w in x]) / len(x))
    df['unique_ratio'] = df.words.apply(lambda x: len(set(x)) / len(x))
    
    # Determine most frequent words by author excluding stopwords
    df['freq_nonstops_counts'] = df.words.apply(lambda x: Counter([w for w in x if w not in stopwords.words('english')]).most_common(5))
    df['freq_nonstops'] = df.freq_nonstops_counts.apply(lambda x: [element[0] for element in x])
    df = df.drop(['freq_nonstops_counts'], axis = 1)
    
    return df

Perform feature extraction on per-article and per-author datasets

In [7]:
articles_aggr = extract_features(articles_aggr)
articles = extract_features(articles)
articles = articles.drop(['freq_nonstops'], axis = 1)   # We won't need most frequent words on a per-article basis

I thought looking at most frequent bigrams or trigrams would be valuable but they turn out to carry too much semantic meaning as opposed to syntactic. They would be helpful but the point of the model is not to figure out who writes about the Czech Republic but to differentiate between innate writing styles

In [8]:
example_text = articles_aggr.loc[1,'text']
ngram_counts = Counter(ngrams(example_text.split(), 3))
ngram_counts.most_common(10)

[(('the', 'prague', 'stock'), 42),
 (('--', 'prague', 'newsroom,'), 41),
 (('prague', 'newsroom,', '42-2-2423-0003'), 41),
 (('in', 'the', 'second'), 38),
 (('the', 'central', 'bank'), 38),
 (('a', 'lot', 'of'), 31),
 (('but', 'analysts', 'said'), 28),
 (('in', 'the', 'first'), 28),
 (('prague', 'stock', 'exchange'), 27),
 (('the', 'czech', 'republic'), 25)]

## 3) Feedforward network

To determine whether a work was completed by a candidate, I use a simple feed-forward neural net to classify a match or no-match. For each article written, I present multiple candidates for authorship and the network tries to determine correctly which is the correct author

In [9]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from math import ceil
import pandas as pd
import numpy as np

# Duplicated the data many times and then present a random candidate for authorship
# There is a very small chance of getting duplicates (about 4% of the merge), but duplicates are deleted below
ten_X_data = pd.concat([articles, articles, articles, articles, articles, articles, articles, articles, articles, articles], axis = 0, ignore_index = True)
ten_X_data = ten_X_data.drop(['sentences'], axis = 1)
sum_stats = articles_aggr
sum_stats = sum_stats.rename(columns= {'author':'candidate','sent_wc':'sent_wc_cand','sent_punc':'sent_punc_cand','avg_wlength':'avg_wlength_cand','unique_ratio':'unique_ratio_cand'})
sum_stats = sum_stats.drop(['text','sentences','words'], axis = 1)

# The aggregate dataset is much smaller than the article-specific one, so we exponentially grow it to be able to merge it
doubling = 0
while doubling < 10:
    sum_stats = pd.concat([sum_stats, sum_stats], axis = 0, ignore_index = True)
    doubling += 1

# Merge article data with candidate, aggregate data
paired_data = pd.concat([ten_X_data, sum_stats.sample(frac = 1).reset_index(drop=True)[:len(ten_X_data)]], axis = 1)
paired_data = paired_data.drop_duplicates(subset = ['text','candidate'])

# Frequency of candidate's most used words in the article
paired_data['common_words'] = paired_data.apply(lambda row: len([w for w in row['words'] if w in row['freq_nonstops']]) / len(row['words']), axis = 1)
# Target variable (whether candidate is actually the author)
paired_data['match'] = (paired_data.author == paired_data.candidate)

# Balance the target variable to be about 50/50 (without this, we only have about 2% of matches)
paired_data = paired_data.sort_values(by = ['match'])
paired_data['del'] = pd.Series(np.random.rand(len(paired_data.match))) - paired_data['match'] 
paired_data = paired_data[paired_data['del'] < 0.02]

# re-order and drop some columns
cols = ['sent_wc','sent_punc','avg_wlength','unique_ratio','sent_wc_cand','sent_punc_cand','avg_wlength_cand','unique_ratio_cand','common_words','match','text','author','candidate']
paired_data = paired_data[cols]

# Standardize input data
normalized = paired_data.copy()
normalized.iloc[:,:9] = (normalized.iloc[:,:9] - normalized.iloc[:,:9].mean()) / normalized.iloc[:,:9].std()

# Split into train and test sets. Validation set split during training
shuffled = normalized.sample(frac = 1).reset_index(drop = True)
split = ceil(0.8*len(shuffled))
train_data = shuffled.iloc[:split,:]
test_data = shuffled.iloc[split:,:]
X_train = train_data.iloc[:,:9].values
y_train = train_data.iloc[:,9].values

test_data = shuffled.iloc[split:,:]
X_test = test_data.iloc[:,:9].values
y_test = test_data.iloc[:,9].values

C:\Users\patri\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Now the data is ready and we can build and train the neural net

In [10]:
lr = 0.04

model = Sequential()
model.add(Dense(20, input_shape = (9,), activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense((10), activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense((1), activation = 'sigmoid'))

model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x = X_train, y = y_train, validation_split = 0.3, epochs = 500, verbose = 1)

Train on 907 samples, validate on 389 samples
Epoch 1/500
907/907 [==============================] - 3s 4ms/step - loss: 0.5822 - acc: 0.7045 - val_loss: 0.5396 - val_acc: 0.7558
Epoch 2/500
907/907 [==============================] - 0s 500us/step - loss: 0.5571 - acc: 0.7453 - val_loss: 0.5173 - val_acc: 0.7815
Epoch 3/500
907/907 [==============================] - 0s 517us/step - loss: 0.5172 - acc: 0.7574 - val_loss: 0.4623 - val_acc: 0.7866
Epoch 4/500
907/907 [==============================] - 1s 551us/step - loss: 0.4981 - acc: 0.7740 - val_loss: 0.4746 - val_acc: 0.7789
Epoch 5/500
907/907 [==============================] - 0s 448us/step - loss: 0.5166 - acc: 0.7740 - val_loss: 0.4993 - val_acc: 0.7918
Epoch 6/500
907/907 [==============================] - 0s 465us/step - loss: 0.5038 - acc: 0.7806 - val_loss: 0.4562 - val_acc: 0.7866
Epoch 7/500
907/907 [==============================] - 0s 345us/step - loss: 0.5151 - acc: 0.7828 - val_loss: 0.4705 - val_acc: 0.7892
Epoch 8/500

Epoch 61/500
907/907 [==============================] - 0s 482us/step - loss: 0.4295 - acc: 0.8049 - val_loss: 0.4846 - val_acc: 0.7841
Epoch 62/500
907/907 [==============================] - 0s 482us/step - loss: 0.4115 - acc: 0.8137 - val_loss: 0.5010 - val_acc: 0.7686
Epoch 63/500
907/907 [==============================] - 0s 517us/step - loss: 0.4430 - acc: 0.8071 - val_loss: 0.5011 - val_acc: 0.7661
Epoch 64/500
907/907 [==============================] - ETA: 0s - loss: 0.4270 - acc: 0.816 - 0s 500us/step - loss: 0.4208 - acc: 0.8203 - val_loss: 0.5512 - val_acc: 0.7506
Epoch 65/500
907/907 [==============================] - 0s 534us/step - loss: 0.4079 - acc: 0.8192 - val_loss: 0.5865 - val_acc: 0.7763
Epoch 66/500
907/907 [==============================] - 0s 482us/step - loss: 0.4338 - acc: 0.8181 - val_loss: 0.4886 - val_acc: 0.7892
Epoch 67/500
907/907 [==============================] - 0s 482us/step - loss: 0.4304 - acc: 0.8071 - val_loss: 0.5637 - val_acc: 0.7686
Epoch 68/5

Epoch 121/500
907/907 [==============================] - 1s 569us/step - loss: 0.4326 - acc: 0.8137 - val_loss: 0.5626 - val_acc: 0.7789
Epoch 122/500
907/907 [==============================] - 1s 620us/step - loss: 0.4169 - acc: 0.8236 - val_loss: 0.5538 - val_acc: 0.7763
Epoch 123/500
907/907 [==============================] - 0s 534us/step - loss: 0.4189 - acc: 0.8247 - val_loss: 0.5410 - val_acc: 0.7815
Epoch 124/500
907/907 [==============================] - 0s 482us/step - loss: 0.3954 - acc: 0.8313 - val_loss: 0.5402 - val_acc: 0.7815
Epoch 125/500
907/907 [==============================] - 0s 500us/step - loss: 0.4225 - acc: 0.8148 - val_loss: 0.5188 - val_acc: 0.7943
Epoch 126/500
907/907 [==============================] - 0s 534us/step - loss: 0.4233 - acc: 0.8148 - val_loss: 0.5955 - val_acc: 0.7892
Epoch 127/500
907/907 [==============================] - 1s 551us/step - loss: 0.3949 - acc: 0.8291 - val_loss: 0.5567 - val_acc: 0.7892
Epoch 128/500
907/907 [==================

Epoch 181/500
907/907 [==============================] - 0s 500us/step - loss: 0.4173 - acc: 0.8225 - val_loss: 0.5695 - val_acc: 0.7918
Epoch 182/500
907/907 [==============================] - 0s 362us/step - loss: 0.3787 - acc: 0.8258 - val_loss: 0.6632 - val_acc: 0.7841
Epoch 183/500
907/907 [==============================] - 0s 517us/step - loss: 0.4016 - acc: 0.8357 - val_loss: 0.6376 - val_acc: 0.7866
Epoch 184/500
907/907 [==============================] - 0s 465us/step - loss: 0.3746 - acc: 0.8401 - val_loss: 0.6654 - val_acc: 0.7892
Epoch 185/500
907/907 [==============================] - 0s 534us/step - loss: 0.4443 - acc: 0.8148 - val_loss: 0.5800 - val_acc: 0.7763
Epoch 186/500
907/907 [==============================] - 0s 465us/step - loss: 0.4046 - acc: 0.8269 - val_loss: 0.5340 - val_acc: 0.7892
Epoch 187/500
907/907 [==============================] - 0s 482us/step - loss: 0.4224 - acc: 0.8236 - val_loss: 0.5914 - val_acc: 0.7789
Epoch 188/500
907/907 [==================

907/907 [==============================] - 0s 482us/step - loss: 0.4046 - acc: 0.8269 - val_loss: 0.7058 - val_acc: 0.7738
Epoch 300/500
907/907 [==============================] - 0s 517us/step - loss: 0.4191 - acc: 0.8247 - val_loss: 0.7811 - val_acc: 0.7686
Epoch 301/500
907/907 [==============================] - 0s 465us/step - loss: 0.4182 - acc: 0.8115 - val_loss: 0.7824 - val_acc: 0.7763
Epoch 302/500
907/907 [==============================] - 0s 500us/step - loss: 0.4241 - acc: 0.8126 - val_loss: 0.7451 - val_acc: 0.7738
Epoch 303/500
907/907 [==============================] - 0s 482us/step - loss: 0.4177 - acc: 0.8214 - val_loss: 0.7331 - val_acc: 0.7661
Epoch 304/500
907/907 [==============================] - 0s 534us/step - loss: 0.4123 - acc: 0.8137 - val_loss: 0.7000 - val_acc: 0.7661
Epoch 305/500
907/907 [==============================] - 0s 517us/step - loss: 0.3880 - acc: 0.8236 - val_loss: 0.7752 - val_acc: 0.7712
Epoch 306/500
907/907 [==============================] 

Epoch 359/500
907/907 [==============================] - 1s 620us/step - loss: 0.3897 - acc: 0.8368 - val_loss: 0.7468 - val_acc: 0.7661
Epoch 360/500
907/907 [==============================] - 0s 534us/step - loss: 0.3874 - acc: 0.8280 - val_loss: 0.7636 - val_acc: 0.7686
Epoch 361/500
907/907 [==============================] - 1s 569us/step - loss: 0.4202 - acc: 0.8258 - val_loss: 0.7021 - val_acc: 0.7763
Epoch 362/500
907/907 [==============================] - 0s 482us/step - loss: 0.4162 - acc: 0.8170 - val_loss: 0.7489 - val_acc: 0.7584
Epoch 363/500
907/907 [==============================] - 1s 551us/step - loss: 0.4319 - acc: 0.8170 - val_loss: 0.6449 - val_acc: 0.7738
Epoch 364/500
907/907 [==============================] - 0s 517us/step - loss: 0.4082 - acc: 0.8269 - val_loss: 0.7483 - val_acc: 0.7609
Epoch 365/500
907/907 [==============================] - 0s 500us/step - loss: 0.4050 - acc: 0.8280 - val_loss: 0.7674 - val_acc: 0.7763
Epoch 366/500
907/907 [==================

Epoch 419/500
907/907 [==============================] - 0s 482us/step - loss: 0.4215 - acc: 0.8170 - val_loss: 0.7322 - val_acc: 0.7738
Epoch 420/500
907/907 [==============================] - 0s 482us/step - loss: 0.3954 - acc: 0.8225 - val_loss: 0.8004 - val_acc: 0.7789
Epoch 421/500
907/907 [==============================] - 0s 482us/step - loss: 0.4131 - acc: 0.8236 - val_loss: 0.7356 - val_acc: 0.7866
Epoch 422/500
907/907 [==============================] - 0s 448us/step - loss: 0.4052 - acc: 0.8104 - val_loss: 0.8138 - val_acc: 0.7738
Epoch 423/500
907/907 [==============================] - 0s 500us/step - loss: 0.4147 - acc: 0.8004 - val_loss: 0.7896 - val_acc: 0.7661
Epoch 424/500
907/907 [==============================] - 0s 534us/step - loss: 0.4053 - acc: 0.8302 - val_loss: 0.7730 - val_acc: 0.7738
Epoch 425/500
907/907 [==============================] - 0s 500us/step - loss: 0.4130 - acc: 0.8291 - val_loss: 0.7920 - val_acc: 0.7661
Epoch 426/500
907/907 [==================

Epoch 479/500
907/907 [==============================] - 0s 534us/step - loss: 0.4175 - acc: 0.8236 - val_loss: 0.7017 - val_acc: 0.7892
Epoch 480/500
907/907 [==============================] - 0s 482us/step - loss: 0.3873 - acc: 0.8357 - val_loss: 0.7743 - val_acc: 0.7841
Epoch 481/500
907/907 [==============================] - 0s 500us/step - loss: 0.3972 - acc: 0.8280 - val_loss: 0.7626 - val_acc: 0.7866
Epoch 482/500
907/907 [==============================] - 0s 465us/step - loss: 0.3918 - acc: 0.8456 - val_loss: 0.7568 - val_acc: 0.7943
Epoch 483/500
907/907 [==============================] - 0s 465us/step - loss: 0.3817 - acc: 0.8346 - val_loss: 0.8205 - val_acc: 0.7866
Epoch 484/500
907/907 [==============================] - 0s 482us/step - loss: 0.3973 - acc: 0.8335 - val_loss: 0.8000 - val_acc: 0.7866
Epoch 485/500
907/907 [==============================] - 0s 482us/step - loss: 0.3855 - acc: 0.8335 - val_loss: 0.7629 - val_acc: 0.7866
Epoch 486/500
907/907 [==================

Let's see how well we did

In [11]:
model.evaluate(x = X_test, y = y_test)

323/323 [==============================] - 0s 339us/step


[0.6760535819611683, 0.7554179568408812]

The model achieves 76% test accurracy, which is surprisingly good for the little amount of data we have, and basic enough features that we fed into the algorithm

With enough time, a more appropriate network would probably have been a siamese network with shared weights to differentiate objectively between two writing styles

## LSTM model

The feed-forward net looks at global syntactic features like average word count per sentence. It does a good job, but it captures none of the more dynamic syntactic differences in writing. For this, I tried training a Long Short Term Memory model

In [24]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping
from keras.layers import Flatten
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

Get list of all words to train an embedding vector to

In [13]:
all_text = list(articles_aggr.words)

In [14]:
# Embed words to vectors by training a Word2Vec embedding
model2 = Word2Vec(all_text, min_count = 1, size = 30)

In [15]:
# Get differently formatted list of all words to run tokenizer on
all_words = []
for i in range(len(articles)):
    all_words += articles.loc[i,'words']
all_words = list(set(all_words))

In [16]:
tokenizer = Tokenizer()   # Numeric tokenizer
tokenizer.fit_on_texts([' '.join(all_words)])

Manipulate the text data and get it ready to feed into the LSTM

In [17]:
texts = articles.copy()
texts['keras_tokens'] = texts.words.apply(lambda x: [tokenizer.word_index[w] for w in x])  # Numeric tokens

# For LSTMs, each input sentence has to be the same length, so we pad shorter sentences with zeros
texts['padded'] = pad_sequences(list(texts.keras_tokens), maxlen = 1200).tolist()
inv_mapping = {k:v for v,k in tokenizer.word_index.items()}   # Inverse dictionary from numbers back to words

# Embed words using previously trained Word2Vec model
texts['embedded'] = texts.padded.apply(lambda x: [model2[inv_mapping[w]] if w != 0 else np.zeros((30,)) for w in x])

C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


Train LSTM model

In [30]:
authors = list(articles_aggr.author)
authors[0]

'AaronPressman'

Here will train a RNN for the first author of the list. A similar RNN could be trained with the other authors, with some slight hyperparameter tuning.

In [31]:
pov_data = texts.copy()
pov_data['target'] = pov_data['author'] == authors[0]    # Set target to 1 if article author is candidate author

pov_data['del'] = pd.Series(np.random.rand(len(pov_data.target))) - pov_data['target']    # Delete some of the non-match cases
pov_data = pov_data[pov_data['del'] < 0.04]

pov_data = pov_data.drop(['text', 'sentences', 'words', 'keras_tokens', 'padded', 'del'], axis = 1)

shuffled = pov_data.sample(frac = 1).reset_index(drop = True)
split = ceil(0.8*len(shuffled))
train_data = shuffled.iloc[:split,:]
test_data = shuffled.iloc[split:,:]
X_train = train_data.loc[:,'embedded'].values
y_train = train_data.loc[:,'target'].values

test_data = shuffled.iloc[split:,:]
X_test = test_data.loc[:,'embedded'].values
y_test = test_data.loc[:,'target'].values

# Reshape input variables
for i in range(len(X_test)):
    X_test[i] = np.vstack(tuple(X_test[i]))
    X_test = np.stack(X_test)

for i in range(len(X_train)):
    X_train[i] = np.vstack(tuple(X_train[i]))
    X_train = np.stack(X_train)


lr = 0.0002    # Learning rate for the algorithm

# Build and train the model using one hidden layer of LSTM cells and two dense layer feeding to binary target
model = Sequential()
model.add(LSTM(batch_input_shape=(None,1200, 30), units = 40, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))
model.add(TimeDistributed(Dense(16, activation = 'relu')))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x = X_train, y = y_train, validation_split = 0.2, callbacks = [EarlyStopping(monitor = 'val_loss', patience = 6)], epochs = 40, verbose = 2)

Train on 191 samples, validate on 48 samples
Epoch 1/40
 - 12s - loss: 0.6373 - acc: 0.6178 - val_loss: 0.5773 - val_acc: 0.7083
Epoch 2/40
 - 4s - loss: 0.6037 - acc: 0.6597 - val_loss: 0.5647 - val_acc: 0.7083
Epoch 3/40
 - 4s - loss: 0.5827 - acc: 0.6597 - val_loss: 0.5584 - val_acc: 0.7083
Epoch 4/40
 - 4s - loss: 0.5508 - acc: 0.6649 - val_loss: 0.5560 - val_acc: 0.7083
Epoch 5/40
 - 4s - loss: 0.5430 - acc: 0.6859 - val_loss: 0.5354 - val_acc: 0.7083
Epoch 6/40
 - 4s - loss: 0.5110 - acc: 0.6859 - val_loss: 0.5136 - val_acc: 0.7083
Epoch 7/40
 - 4s - loss: 0.4823 - acc: 0.7539 - val_loss: 0.4974 - val_acc: 0.7708
Epoch 8/40
 - 4s - loss: 0.4645 - acc: 0.7592 - val_loss: 0.4802 - val_acc: 0.7708
Epoch 9/40
 - 4s - loss: 0.4304 - acc: 0.7958 - val_loss: 0.4511 - val_acc: 0.7708
Epoch 10/40
 - 4s - loss: 0.3922 - acc: 0.8639 - val_loss: 0.4401 - val_acc: 0.7917
Epoch 11/40
 - 4s - loss: 0.3558 - acc: 0.8848 - val_loss: 0.3954 - val_acc: 0.8125
Epoch 12/40
 - 5s - loss: 0.4328 - acc:

In [32]:
model.evaluate(x = X_test, y = y_test)

59/59 [==============================] - 0s 6ms/step


[0.5898911553924366, 0.813559310921168]

This model does remarkebly well (81% test accuracy), especially given how little data we have. With more data and more careful parameter tuning this model would do very, very well in verifying authorship!

There are many kinds of applications for this model. It can be used by teachers in schools to ensure students are submitting their own work. It can also be used to determine the authorship of unknown works. I assume that in grade school, it would perform even better. In the current dataset, we have writings of proessional journalists who work for the same company. At say the sixth grade level, where students can be writing at very different levels, I suspect the model would do even better at verifying authorship.

Also, if we trained this model for different authors and found that it does better at verifying some authors as opposed to others, this could be argued to be an indication of how unique a writing style is